In [2]:
import os
import math
import string
from PIL import Image, ImageDraw, ImageFont
from google.colab import drive
from google.colab import files  # Biblioteca para download automático

# 1. Montar o Google Drive
drive.mount('/content/drive', force_remount=True)

def criar_colagem_cientifica_700dpi(diretorio_alvo, nome_saida='colagem_final_700dpi.jpg', colunas=4):

    # Limpeza e verificação do caminho
    diretorio_alvo = diretorio_alvo.strip()
    if not os.path.exists(diretorio_alvo):
        print(f"❌ ERRO: Diretório não encontrado: {diretorio_alvo}")
        return

    # Extensões permitidas
    extensoes = ('.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.tif')
    arquivos = [f for f in os.listdir(diretorio_alvo) if f.lower().endswith(extensoes)]
    arquivos.sort(key=lambda x: x.lower()) # Ordem alfabética

    if not arquivos:
        print("⚠️ Nenhuma imagem encontrada.")
        return

    # Selecionar até 24 imagens
    lista_img = arquivos[:24]
    print(f"Processando {len(lista_img)} imagens para saída em 700 DPI...")

    # 2. Determinar a resolução máxima (Pixel Dimensions)
    # Para garantir 700 DPI com qualidade, precisamos de muitos pixels.
    # O código buscará a maior largura e altura disponíveis nas suas amostras.
    imagens_obj = []
    max_w = 0
    max_h = 0

    for nome in lista_img:
        caminho = os.path.join(diretorio_alvo, nome)
        try:
            # Abre e força RGB (preserva cores)
            img = Image.open(caminho).convert('RGB')
            imagens_obj.append(img)

            if img.width > max_w: max_w = img.width
            if img.height > max_h: max_h = img.height
        except Exception as e:
            print(f"Erro ao ler {nome}: {e}")

    if not imagens_obj:
        return

    # Configurar tamanho da célula da grade
    largura_celula = max_w
    altura_celula = max_h

    # Calcular tamanho total do Canvas
    num_imgs = len(imagens_obj)
    linhas = math.ceil(num_imgs / colunas)

    largura_total = colunas * largura_celula
    altura_total = linhas * altura_celula

    print(f"Resolução do Canvas: {largura_total}x{altura_total} pixels")

    # Criar Canvas Branco
    canvas = Image.new('RGB', (largura_total, altura_total), (255, 255, 255))

    # Configurar Fonte
    tamanho_fonte = int(altura_celula * 0.12) # Fonte proporcional
    try:
        fonte = ImageFont.truetype("LiberationSans-Regular.ttf", tamanho_fonte)
    except:
        fonte = ImageFont.load_default()

    alfabeto = string.ascii_uppercase

    # 3. Montagem
    for i, img in enumerate(imagens_obj):
        col = i % colunas
        row = i // colunas
        x = col * largura_celula
        y = row * altura_celula

        # Redimensionamento suave (LANCZOS) para o tamanho da célula
        img_final = img.resize((largura_celula, altura_celula), Image.LANCZOS)
        canvas.paste(img_final, (x, y))

        # Etiquetagem (Letra com fundo preto)
        draw = ImageDraw.Draw(canvas)
        letra = alfabeto[i] if i < 26 else f"Z{i}"

        margem = int(largura_celula * 0.03)
        pos_txt = (x + margem, y + margem)

        bbox = draw.textbbox(pos_txt, letra, font=fonte)
        draw.rectangle((bbox[0]-10, bbox[1]-10, bbox[2]+10, bbox[3]+10), fill="black")
        draw.text(pos_txt, letra, fill="white", font=fonte)

    # 4. Salvamento com Metadados DPI
    caminho_final = os.path.join(diretorio_alvo, nome_saida)

    # dpi=(700, 700) insere o metadado solicitado
    # subsampling=0 previne a compressão de cor do JPEG (chroma subsampling)
    canvas.save(caminho_final, format='JPEG', quality=100, subsampling=0, dpi=(700, 700))

    print(f"✅ Arquivo salvo no Drive: {caminho_final}")
    print("⏳ Iniciando download para o seu computador...")

    # 5. Download Automático
    files.download(caminho_final)

# --- EXECUÇÃO ---
pasta_alvo = '/content/drive/MyDrive/pesquisa_mestrado_2025/unir_fotos'
criar_colagem_cientifica_700dpi(pasta_alvo)

Mounted at /content/drive
Processando 24 imagens para saída em 700 DPI...
Resolução do Canvas: 11628x13716 pixels
✅ Arquivo salvo no Drive: /content/drive/MyDrive/pesquisa_mestrado_2025/unir_fotos/colagem_final_700dpi.jpg
⏳ Iniciando download para o seu computador...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>